In [58]:
import requests
import pandas as pd
from datetime import datetime, timedelta

YEARS_FAR_BACK = 25
END_DATE = datetime.now().strftime("%Y-%m-%d")
START_DATE = (datetime.now() - timedelta(days=YEARS_FAR_BACK*365)).strftime("%Y-%m-%d")
PAGE_SIZE = 1000

def get_fiscal_data(endpoint, params):
    base_url = "https://api.fiscaldata.treasury.gov/services/api/fiscal_service"
    url = f"{base_url}/{endpoint}"
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()['data']
    else:
        raise Exception(f"API request failed with status code {response.status_code}")

In [235]:
def get_treasury_data():
    # Initialize an empty list to hold data
    all_debt_data = []
    all_exp_rev_data = []

    # Iterate through pages 1 to 100
    for page_number in range(1, 8):
        # Fetch debt data
        debt_data = get_fiscal_data(
            "v2/accounting/od/debt_to_penny",
            {
                "fields": "record_date,tot_pub_debt_out_amt",
                "filter": f"record_date:gte:{START_DATE},record_date:lte:{END_DATE}",
                "sort": "-record_date",
                "page[number]": page_number,
                "page[size]": PAGE_SIZE,
            }
        )
        all_debt_data.extend(debt_data)
        # Fetch expenditure and revenue data

    for desc in ["Total Outlays", "Total Receipts", "Surplus (+) or Deficit (-)"]:
        monthly_finances = get_fiscal_data(
                    "/v1/accounting/mts/mts_table_3",
                    {
                        "fields": "record_date,current_month_rcpt_outly_amt, classification_desc",#, current_month_gross_outly_amt, current_month_dfct_sur_amt",
                        "filter": f"record_date:gte:{START_DATE},record_date:lte:{END_DATE},classification_desc:eq:{desc}",
                        "sort": "-record_date",
                        "page[number]": 1,
                        "page[size]": 1000,
                    }
                )
        all_exp_rev_data.extend(monthly_finances)
    debt_data_df = pd.DataFrame(all_debt_data)
    monthly_df = pd.DataFrame(all_exp_rev_data)
    return debt_data_df, monthly_df


def process_treasury_data(debt_data_df, monthly_df):
    # Convert the record_date column to a datetime object
    debt_data_df['record_date'] = pd.to_datetime(debt_data_df['record_date'])
    monthly_df['record_date'] = pd.to_datetime(monthly_df['record_date'])

    pivoted_df = monthly_df.drop_duplicates().pivot(index='record_date',
                                                    columns='classification_desc', values='current_month_rcpt_outly_amt')
    pivoted_df = pivoted_df.reset_index()
    pivoted_df['month-year'] = pd.to_datetime(pivoted_df['record_date']).dt.to_period('M')
    debt_data_df['month-year'] = pd.to_datetime(debt_data_df['record_date']).dt.to_period('M')
    debt_data_df = debt_data_df.groupby('month-year').first().reset_index()
    debt_data_df = debt_data_df.drop(columns=['record_date'])
    pivoted_df = pivoted_df.drop(columns=['record_date'])

    # Merge the two dataframes
    merged_df = pd.merge(debt_data_df, pivoted_df, on='month-year', how='right')
    return merged_df


In [236]:
debt_data_df, monthly_df = get_treasury_data()
merged_df = process_treasury_data(debt_data_df, monthly_df)

In [238]:
merged_df.tail()

,month-year,tot_pub_debt_out_amt,Surplus (+) or Deficit (-),Total Outlays,Total Receipts
106,2024-01,34191150402393.64,-21930356331.02,499250011788.18,477319655457.16
107,2024-02,34471083238111.75,-296274648079.89,567400716838.98,271126068759.09
108,2024-03,34586533479349.88,-236456823950.20,568547833098.21,332091009148.01
109,2024-04,34616994016250.88,209529306625.33,566668786551.61,776198093176.94
110,2024-05,34667115142400.83,-347130893691.65,670777880670.05,323646986978.40


In [240]:
print("deficit in may 2024")
print(323646986978.40-670777880670.05)
print("debt in may 2024")
print(34616994016250.88--347130893691.65)

deficit in may 2024
-347130893691.65
debt in may 2024
34964124909942.527


In [220]:
pivoted_df = monthly_df.drop_duplicates().pivot(index='record_date', columns='classification_desc', values='current_month_rcpt_outly_amt')
pivoted_df = pivoted_df.reset_index()
pivoted_df.head()

classification_desc,record_date,Surplus (+) or Deficit (-),Total Outlays,Total Receipts
0,2015-03-31,-52910059930.36,287097397834.05,234187337903.69
1,2015-04-30,156714078439.25,315087162351.27,471801240790.52
2,2015-05-31,-82385278419.04,294771105836.73,212385827417.69
3,2015-06-30,51775764001.54,291157289359.24,342933053360.78
4,2015-07-31,-149186766112.85,374680133580.53,225493367467.68


In [122]:
df = pd.read_csv('../data/treasury_data.csv')[['Classification Description',
        'Current Month Gross Receipts Amount',
       'Current Month Gross Outlay Amount',
       'Current Month Deficit Surplus Amount',
       'Fiscal Year', 'Calendar Month Number',
       'Calendar Day Number']]
df

,Classification Description,Current Month Gross Receipts Amount,Current Month Gross Outlay Amount,Current Month Deficit Surplus Amount,Fiscal Year,Calendar Month Number,Calendar Day Number
0,FY 2023,NaN,NaN,NaN,2024,5,31
1,October,3.185001e+11,4.063742e+11,8.787416e+10,2024,5,31
2,November,2.521108e+11,5.006457e+11,2.485349e+11,2024,5,31
3,December,4.549424e+11,5.399431e+11,8.500074e+10,2024,5,31
4,January,4.472883e+11,4.860721e+11,3.878377e+10,2024,5,31
...,...,...,...,...,...,...,...
2494,May,1.998891e+11,3.298600e+11,1.299709e+11,2015,3,31
2495,June,3.236462e+11,2.531274e+11,-7.051872e+10,2015,3,31
2496,July,2.144929e+11,3.091135e+11,9.462055e+10,2015,3,31
2497,August,1.942482e+11,3.229250e+11,1.286768e+11,2015,3,31


In [117]:
21249523565252.10 - 15454459667747.74

5795063897504.361

In [116]:
34616994016250.88 + 5795063897504.36

40412057913755.24

In [ ]:
# extract month-year from record_date
debt_data_df['month-year'] = debt_data_df['record_date'].dt.strftime('%b-%Y')
monthly_df['month-year'] = monthly_df['record_date'].dt.strftime('%b-%Y')

# drop record_date column
debt_data_df.drop_duplicates(subset='month-year', keep='first', inplace=True)
debt_data_df.drop(columns=['record_date'], inplace=True)
monthly_df.drop(columns=['record_date'], inplace=True)

# merge the two dataframes
merged_df = pd.merge(debt_data_df, monthly_df, on='month-year', how='right')

In [111]:
merged_df.head(40)

,tot_pub_debt_out_amt,month-year,current_month_gross_rcpt_amt,current_month_gross_outly_amt,current_month_dfct_sur_amt
0,34667115142400.83,May-2024,15454459667747.74,21249523565252.10,5795063897504.36
1,34616994016250.88,Apr-2024,14807163608428.94,19907965718550.00,5100802110121.06
2,34586533479349.88,Mar-2024,13254791811579.12,18774454276300.80,5519662464721.68
3,34471083238111.75,Feb-2024,12590609793283.10,17637358602581.78,5046748809298.68
4,34191150402393.64,Jan-2024,12048357655764.92,16502557176426.42,4454199520661.50
5,34001493655565.48,Dec-2023,11093718344850.60,15504079875066.94,4410361530216.34
6,33878679367501.20,Nov-2023,10235096854908.74,14386727736095.20,4151630881186.46
7,33699580339128.49,Oct-2023,9685436783346.44,13209043659958.72,3523606876612.28
8,33167334044723.16,Sep-2023,18670805828246.94,24812064490860.42,6141258662613.48
9,32914149363760.95,Aug-2023,17735860461507.02,23535154722370.94,5799294260863.92


In [100]:
merged_df = monthly_df.merge(debt_data_df, on='month-year', how='left')
merged_df.head()

,record_date_x,current_month_gross_rcpt_amt,current_month_gross_outly_amt,current_month_dfct_sur_amt,month-year,record_date_y,tot_pub_debt_out_amt
0,2024-05-31,15454459667747.74,21249523565252.10,5795063897504.36,May-2024,2024-05-31,34667115142400.83
1,2024-05-31,15454459667747.74,21249523565252.10,5795063897504.36,May-2024,2024-05-30,34609796817427.65
2,2024-05-31,15454459667747.74,21249523565252.10,5795063897504.36,May-2024,2024-05-29,34621658355452.18
3,2024-05-31,15454459667747.74,21249523565252.10,5795063897504.36,May-2024,2024-05-28,34606167915025.93
4,2024-05-31,15454459667747.74,21249523565252.10,5795063897504.36,May-2024,2024-05-24,34587980444101.39
